<a href="https://colab.research.google.com/github/Mohamad-Atif1/Flutter_Calculator/blob/master/RPN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## HERE WE START

In [1]:
%matplotlib inline
import torch
from torch.utils.data import Dataset, DataLoader
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
# import pylab
# pylab.rcParams['figure.figsize'] = (8.0, 10.0) # i need to understand thos


In [2]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)


Mounted at /content/drive


In [3]:
dataDir='/content/drive/MyDrive/cocoDataset'
dataType='val2017'
annFile='{}/instances_{}.json'.format(dataDir,dataType)
NEWSIZE = (360,360)
BATCH_SIZE = 8

In [4]:
coco=COCO(annFile)
img_idx = coco.getImgIds()[555]
image = coco.loadImgs(img_idx)
len(io.imread(image[0]['coco_url']))


image[0]['coco_url']

loading annotations into memory...
Done (t=3.22s)
creating index...
index created!


'http://images.cocodataset.org/val2017/000000552371.jpg'

In [5]:
# rpn.RegionProposalNetwork
aaa = []

In [6]:
# x,y,width,height = box
# x2 = x + width
# y2 = y + height

# x = x * wScale / self.image[1]
# x2 = x2 * wScale /self.image[1]
# y = y *  hScale /self.image[0]
#  y2 = y2 * hScale /self.image[0]

In [79]:
class CustomeDataset(Dataset):

    def __init__(self,path,trans=None):
        super().__init__()
        self.coco = COCO(path)
        self.trans = trans

    def __len__(self):
        return len(coco.getImgIds())

    def __getitem__(self,idx):
        img_idx = self.coco.getImgIds()[idx]
        self.image = coco.loadImgs(img_idx)
        self.image = io.imread(self.image[0]['coco_url'])

        hScale = NEWSIZE[0]/self.image.shape[0]
        wScale = NEWSIZE[1]/self.image.shape[1]

        if self.trans:
            self.image = self.trans(self.image)


        annotations_idx = self.coco.getAnnIds(imgIds=img_idx)
        annotations = self.coco.loadAnns(annotations_idx)
        bbox = [self.resize_box(wScale,hScale,annotation['bbox']) for annotation in annotations ]

        # bbox = [annotation['bbox'] for annotation in annotations ]

        bbox = torch.tensor(bbox)
        bbox = bbox.to(device)
        return (self.image,bbox)



    def resize_box(self,wScale,hScale,box):
        # print(type(box),'============================')
        x,y,width,height = box
        x2 = x + width
        y2 = y + height

        x = x * wScale / NEWSIZE[1]
        x2 = x2 * wScale /NEWSIZE[1]
        y = y *  hScale /NEWSIZE[0]
        y2 = y2 * hScale /NEWSIZE[0]

        width = width * wScale
        height =  height * hScale

        return [x,y,x2,y2]


In [61]:
def custom_collate_fn(batch):
    images = []
    targets = []

    for image, target in batch:
        images.append(image)
        dict_tragets = {}
        dict_tragets['boxes'] = target
        dict_tragets['labels'] = torch.ones(len(target),dtype=torch.int64)
        targets.append(dict_tragets)


    # Stack images into a batch
    images = torch.stack(images, dim=0)
    # targets = torch.stack(targets,dim=0)  i cannot do this because image comes with different number of boxes
    #

    return images,targets

In [68]:
import torchvision.transforms as transforms

trans = transforms.Compose([
   transforms.ToTensor(),
   transforms.Resize(NEWSIZE),
#    transforms.CenterCrop(224),
#    transforms.Normalize(mean=[0.485, 0.456, 0.406], # ImageNet
#                         std=[0.229, 0.224, 0.225]),
])
dataset = CustomeDataset(path=annFile,trans=trans)

train_loader = DataLoader(dataset,batch_size=BATCH_SIZE,collate_fn = custom_collate_fn)
dataset[8][1]

loading annotations into memory...
Done (t=0.44s)
creating index...
index created!
cuda:0


/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


tensor([[0.0520, 0.0000, 0.5257, 1.0000],
        [0.3575, 0.4046, 0.6626, 0.8294],
        [0.3693, 0.0271, 0.7892, 0.9174],
        [0.5418, 0.1727, 0.8447, 0.7354],
        [0.7006, 0.0973, 0.9166, 0.4852],
        [0.6248, 0.0667, 0.8600, 0.5660],
        [0.8437, 0.1852, 1.0000, 0.5397],
        [0.0000, 0.0000, 0.3152, 1.0000],
        [0.7879, 0.3190, 0.8701, 0.3994],
        [0.6697, 0.3036, 0.8802, 0.5476]], device='cuda:0')

In [69]:
batch = next(iter(train_loader))

cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0


In [73]:
batch[0].device

device(type='cpu')

In [11]:
# batch[1]


# targets = [
#     {
#         'boxes': torch.tensor([[20, 30, 100, 120], [50, 60, 150, 140]]),
#         'labels': torch.tensor([1, 2]),
#     },
#     {
#         'boxes': torch.tensor([[30, 40, 120, 130]]),
#         'labels': torch.tensor([1]),
#     }
# ]

In [12]:
# len(batch)
# batch[1]

In [13]:
# batch[0].shape
# aaa[0]

In [14]:
# for batch in train_loader:
#     image,bbox = batch[0],batch[1]
#     print(bbox)




In [15]:
# model = torchvision.models.resnet50()
# model = torch.nn.Sequential(*list(torchvision.models.resnet101(pretrained=True).children())[:-3])
# model
# model(batch[0]).shape

In [16]:
# import torch
# import torchvision.models
# from torchvision.models.detection.rpn import RegionProposalNetwork
# from torchvision.models.detection.rpn import AnchorGenerator
# from torchvision.models.detection.rpn import RPNHead
# from torchvision.models.detection import fasterrcnn_resnet50_fpn



# # Define the RPN model
# rpn = RegionProposalNetwork(
#     anchor_generator=AnchorGenerator(sizes=((32,), (64,), (128,))),
#     head=RPNHead(256,9),
#     fg_iou_thresh=0.7,
#     bg_iou_thresh=0.3,
#     batch_size_per_image=256,
#     positive_fraction=0.5,
#     pre_nms_top_n=1000,
#     post_nms_top_n=1000,
#     nms_thresh=0.7,
# )


# # Define the loss function
# criterion = torch.nn.BCELoss()

# # Define the optimizer
# optimizer = torch.optim.Adam(rpn.parameters(), lr=0.001)

# # model.eval()

# # Train the model
# for epoch in range(10):
#   for images, bboxes in train_loader:
#     # Feed the images and bboxes to the model
#     f = model(images)
#     predictions = rpn(images,f)

#     # Calculate the loss
#     loss = criterion(predictions, bboxes)

#     # Update the model parameters
#     optimizer.zero_grad()
#     loss.backward()
#     optimizer.step()
#     print("one step done!")
#     break

# # Evaluate the model
# # ...




In [17]:
# import torch
# from torchvision.models.detection import fasterrcnn_resnet50_fpn
# from torchvision.models.detection.rpn import RegionProposalNetwork

# # Load the ResNet model
# resnet = fasterrcnn_resnet50_fpn(pretrained=True)

# # Load the RPN model
# rpn = RegionProposalNetwork(
#     anchor_generator=AnchorGenerator(sizes=((32,), (64,), (128,))),
#     head=RPNHead(256),
#     fg_iou_thresh=0.7,
#     bg_iou_thresh=0.3,
#     batch_size_per_image=256,
#     positive_fraction=0.5,
#     pre_nms_top_n=1000,
#     post_nms_top_n=1000,
#     nms_thresh=0.7,
# )

# # Load the image
# image = torch.randn(1, 3, 224, 224)

# # Extract the features from the image using ResNet
# features = resnet.features(image)

# # Feed the features to the RPN model
# predictions = rpn(features)

# # Process the predictions


In [18]:
# # #FASTRCNN

# model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
# # For training
# images, boxes = torch.rand(8, 3, 600, 1200), torch.rand(8, 11, 4)
# boxes[:, :, 2:4] = boxes[:, :, 0:2] + boxes[:, :, 2:4]
# labels = torch.randint(1, 91, (8, 11))
# images = list(image for image in images)
# targets = []
# for i in range(len(images)):
#      d = {}
#      d['boxes'] = boxes[i]
#     #  d['labels'] = labels[i]
#      targets.append(d)

# output = model(images, targets)
# # # >>> # For inference
# # # >>> model.eval()
# # # >>> x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
# # # >>> predictions = model(x)
# # # >>>
# # # >>> # optionally, if you want to export the model to ONNX:
# # # >>> torch.onnx.export(model, x, "faster_rcnn.onnx", opset_version = 11)

In [19]:
# targets

In [20]:
# boxes.shape

In [21]:
# targets[0]

In [22]:
# output

In [23]:
# import torch
# import torchvision
# from torchvision.models.detection.rpn import AnchorGenerator

# # Load the pre-trained Faster R-CNN model
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# # Remove the classification and regression heads
# model.roi_heads.box_predictor = None

# # Create an RPN-only model by extracting the RPN module
# rpn_model = model.rpn

# # Set the RPN model to evaluation mode
# rpn_model.train()

# # Define an input tensor (image)
# input_image = torch.randn(1, 3, 224, 224)  # Example input image

# ground_truth_boxes = [torch.tensor([[20, 20, 100, 100], [50, 50, 150, 150]]),  # Example ground truth bounding boxes for image 1
#                       torch.tensor([[30, 30, 120, 120], [60, 60, 140, 140]])]  # Example ground truth bounding boxes for image 2
# labels = [torch.tensor([1, 1]), torch.tensor([1, 1])]
# # Generate region proposals using the RPN
# backbone_features = model.backbone(input_image)
# proposals, _ = rpn_model(input_image,backbone_features,ground_truth_boxes)

# print(proposals)

In [24]:
# rpn_model

In [25]:
# import torch
# import torchvision
# from torchvision.models.detection.rpn import AnchorGenerator
# from torchvision.models.detection.image_list import ImageList

# # Load the pre-trained Faster R-CNN model
# model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# # Remove the classification and regression heads
# model.roi_heads.box_predictor = None

# # Create an RPN-only model by extracting the RPN module
# rpn_model = model.rpn

# # Set the RPN model to evaluation mode
# rpn_model.eval()

# # Define an input tensor (image)
# input_image = torch.randn(2, 3, 224, 224)  # Example input image

# # Wrap the input tensor in an ImageList object
# image_list = ImageList(input_image, [(input_image.shape[-2], input_image.shape[-1])])

# # Get features from the backbone network
# backbone_features = model.backbone(image_list.tensors)

# # Generate region proposals using the RPN
# proposals, _ = rpn_model(image_list, backbone_features)

# print(proposals)

In [26]:
import torch
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.image_list import ImageList


#DONT TOUCH IT , IT WORKS!!


# Load the pre-trained Faster R-CNN model
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
# Set the RPN model to training mode
model.train()
# Remove the classification and regression heads

# model.roi_heads.box_predictor = None # CHECK THIS OUT!! double check !!!!!!!!!

# Create an RPN-only model by extracting the RPN module
# rpn_model = model.rpn.head()
rpn_model = model.rpn
#Non max score
# model.rpn.score_thresh = 0.7 # i need to understands this more by watching non max

# Set the RPN model to evaluation mode
rpn_model.train()
# Example input image
input_image = batch[0]

# Wrap the input tensor in an ImageList object
image_list = ImageList(input_image, [(input_image.shape[-2], input_image.shape[-1])] * input_image.shape[0])

# Get features from the backbone network
backbone_features = model.backbone(image_list.tensors)

# Generate region proposals using the RPN
proposals, loss = rpn_model(image_list, backbone_features,batch[1])

# print(proposals)
loss

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth
100%|██████████| 160M/160M [00:01<00:00, 103MB/s]


{'loss_objectness': tensor(2.1227, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>),
 'loss_rpn_box_reg': tensor(1.0790, grad_fn=<DivBackward0>)}

In [27]:
# objectness[4].shape

In [28]:
model.eval()
output = model(batch[0],batch[1])


In [29]:
output[5]['boxes'].shape

torch.Size([10, 4])

In [30]:
# model.rpn.nms_thresh

In [31]:
batch[1][7]['boxes']*NEWSIZE[0] # unnormalized

# apply non max suppression to the proposed regions
len(proposals[0])

2000

In [32]:
proposals[7].shape


torch.Size([2000, 4])

## Training

In [33]:
model.rpn

RegionProposalNetwork(
  (anchor_generator): AnchorGenerator()
  (head): RPNHead(
    (conv): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): ReLU(inplace=True)
      )
    )
    (cls_logits): Conv2d(256, 3, kernel_size=(1, 1), stride=(1, 1))
    (bbox_pred): Conv2d(256, 12, kernel_size=(1, 1), stride=(1, 1))
  )
)

In [76]:
image_list.tensors.device

device(type='cuda', index=0)

In [78]:
import torch.optim as opt

rpn_model.train()
params_to_optimize = []
params_to_optimize += list(rpn_model.parameters())
params_to_optimize += list(model.backbone.parameters())

optimizer = opt.SGD(params_to_optimize,lr=0.001, momentum=0.9, weight_decay=0.0005)
EPOCHS = 1
train_loss = []

device = torch.device('cuda:0')
backbone = model.backbone.to(device)
rpn_model = rpn_model.to(device)
for i in range(EPOCHS):
    loss = 0
    for i,batch in enumerate(train_loader):
        print(i)
        image, bbox = batch
        image = image.to(device)
        # image, bbox = image.to(device),bbox.to(device)
        image_list = ImageList(image, [(image.shape[-2], image.shape[-1])] * image.shape[0]) # imgs,imgs_sizes*batch size
        backbone_features = backbone((image_list.tensors).to(device))
        proposals, losses = rpn_model(image_list, backbone_features,batch[1])
        losses = losses["loss_objectness"] + losses["loss_rpn_box_reg"]
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        loss += losses
    train_loss.append(loss/len(train_loader))




cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
1
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
2
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
3
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
cuda:0
4


KeyboardInterrupt: ignored

In [ ]:
losses

In [ ]:
# rpn_model.eval()
# out = rpn_model(image_list,backbone_features)
# on evaluation mode , we got 1000 preposal regions
# we dont get losses
results = []
for i in range(len(proposals)):
    boxes = torchvision.ops.nms(proposals[i],scores[i],0.1)
    results.append(boxes)

# nm = torchvision.ops.nms(proposals_tensors,scores_tensors,0.5)
# nm
results[0].shape


In [ ]:
bboxes = []
for i in nm:
    bboxes.append(out[0][0][i])

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

image = batch[0][0]
boxes = bboxes
plt.imshow(image.permute(1,2,0))
# print(len(boxes))

# Draw bounding boxes on the image
for box in boxes:

    x, y, x2, y2 = box
    width = (x2-x)
    height = (y2-y)

    plt.gca().add_patch(matplotlib.patches.Rectangle((x,y),width,height,
                    edgecolor='red',
                    facecolor='none',
                    lw=4))


In [ ]:
proposals[0].shape

In [ ]:

image_list.image_sizes

In [ ]:
model.backbone(torch.randn(1,3,224,224))
model.rpn.head.cls_logits(torch.randn(1,256,256,3)).shape

In [ ]:
# model.backbone(torch.randn(1,3,224,224))
# logits = []
# bbox_reg = []
#     for feature in x:
#             t = model.backbone(feature)
#             logits.append(self.cls_logits(t))
#             bbox_reg.append(self.bbox_pred(t))

In [ ]:
# trans = transforms.Compose([
#    transforms.ToTensor(),
# #    transforms.Resize((350,350)),
# #    transforms.CenterCrop(224),
#    transforms.Normalize(mean=[0.485, 0.456, 0.406], # ImageNet
#                         std=[0.229, 0.224, 0.225]),
# ])
# dataset = CustomeDataset(path='/content/datadir/instances_val2017.json' , trans = trans)


In [ ]:
# import cv2
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib

# image, boxes = (dataset[55][0]),dataset[55][1]
# print(image.shape)
# plt.imshow(image.permute(1,2,0))
# # Draw bounding boxes on the image
# for box in boxes:
#     print(box)
#     print("===============")
#     _x, _y, width, height = box
#     print(width)
#     x = _x * wScale
#     y = _y * hScale
#     w = width*wScale
#     h = height * hScale
#     color = (0, 255, 0)
#     plt.gca().add_patch(matplotlib.patches.Rectangle((x,y),w,h,
#                     edgecolor='red',
#                     facecolor='none',
#                     lw=4))
#     # Display the image with bounding boxes
# # cv2.imshow("Image with Bounding Boxes", image)
# # cv2.waitKey(0)
# # cv2.destroyAllWindows()


In [ ]:
# x
# w/width

In [ ]:
# image.shape[1]
# height = 424
# width = 640
# wScale = NEWSIZE[0]/width
# hScale = NEWSIZE[0]/height
# image.shape[1]

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib


image, boxes = (dataset[51][0]),dataset[51][1]
plt.imshow(image.permute(1,2,0))

# Draw bounding boxes on the image
for box in boxes:

    x, y, x2, y2 = box
    width = (x2-x)*NEWSIZE[1]
    height = (y2-y)*NEWSIZE[0]

    plt.gca().add_patch(matplotlib.patches.Rectangle((x*NEWSIZE[1],y*NEWSIZE[0]),width,height,
                    edgecolor='red',
                    facecolor='none',
                    lw=4))


In [ ]:
_,h,w = dataset2[8][0].shape


## Center Crop Source Code

In [ ]:

def center_crop(img: Tensor, output_size: List[int]) -> Tensor:
    """Crops the given image at the center.
    If the image is torch Tensor, it is expected
    to have [..., H, W] shape, where ... means an arbitrary number of leading dimensions.
    If image size is smaller than output size along any edge, image is padded with 0 and then center cropped.

    Args:
        img (PIL Image or Tensor): Image to be cropped.
        output_size (sequence or int): (height, width) of the crop box. If int or sequence with single int,
            it is used for both directions.

    Returns:
        PIL Image or Tensor: Cropped image.
    """
    if not torch.jit.is_scripting() and not torch.jit.is_tracing():
        _log_api_usage_once(center_crop)
    if isinstance(output_size, numbers.Number):
        output_size = (int(output_size), int(output_size))
    elif isinstance(output_size, (tuple, list)) and len(output_size) == 1:
        output_size = (output_size[0], output_size[0])

    _, image_height, image_width = get_dimensions(img)
    crop_height, crop_width = output_size

    if crop_width > image_width or crop_height > image_height:
        padding_ltrb = [
            (crop_width - image_width) // 2 if crop_width > image_width else 0,
            (crop_height - image_height) // 2 if crop_height > image_height else 0,
            (crop_width - image_width + 1) // 2 if crop_width > image_width else 0,
            (crop_height - image_height + 1) // 2 if crop_height > image_height else 0,
        ]
        img = pad(img, padding_ltrb, fill=0)  # PIL uses fill value 0
        _, image_height, image_width = get_dimensions(img)
        if crop_width == image_width and crop_height == image_height:
            return img

    crop_top = int(round((image_height - crop_height) / 2.0))
    crop_left = int(round((image_width - crop_width) / 2.0))
    return crop(img, crop_top, crop_left, crop_height, crop_width)

